In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkb import loads

geo_parquet_file = "/Users/vinhvuainguyen/Geoparquet/hais_2024-04-15.geoparquet"

# Les filen
df = pd.read_parquet("hais_2024-04-15.geoparquet")

# Filtrer kun LOFOTEN og SUPERSPEED 1
df = df[df["ship_name"].isin(["LOFOTEN", "SUPERSPEED 1"])]
print("Etter skipfilter:", len(df))


# Konvertere dato/tid
df["date_time_utc"] = pd.to_datetime(df["date_time_utc"])
df["hour"] = df["date_time_utc"].dt.hour

# Sorter på tid 
df = df.sort_values(by="date_time_utc").reset_index(drop=True)

df["ship_name"] = df["ship_name"].astype(str)  # eller .astype("string")

df.to_parquet(
    "ais_data_partitioned",
    partition_cols=["ship_name", "hour"],
    index=False,
)


import pyarrow.parquet as pq

filters = [
    ("ship_name", "=", "SUPERSPEED 1"),  # valgfritt
    ("hour", ">=", 12),
    ("hour", "<=", 14)
]

table = pq.read_table(
    "ais_data_partitioned",
    filters=filters,
    columns=["date_time_utc", "ship_name", "longitude", "latitude", "geometry"]
)
df_temp = table.to_pandas()

print("Antall rader kl. 12–14:", len(df_temp))


import folium

# Sorter dataene kronologisk
df_sorted = df_temp.sort_values(by="date_time_utc")

# Hent første og siste rad
df_first = df_sorted.head(1)
df_last = df_sorted.tail(1)

# Finn midtpunkt for kartet
center_lat = df_sorted["latitude"].mean()
center_lon = df_sorted["longitude"].mean()


# Visualisering avstand
# Lag Folium-kart
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Plot første punkt (grønn markør)
for _, row in df_first.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Første posisjon: {row['ship_name']} - {row['date_time_utc']}",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)

# Plot siste punkt (rød markør)
for _, row in df_last.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Siste posisjon: {row['ship_name']} - {row['date_time_utc']}",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)

# Tegn linje mellom første og siste punkt
# (Hent ut lat/lon fra df_first og df_last)
start_coords = [df_first["latitude"].iloc[0], df_first["longitude"].iloc[0]]
end_coords = [df_last["latitude"].iloc[0], df_last["longitude"].iloc[0]]
folium.PolyLine(locations=[start_coords, end_coords],
                color="blue",
                weight=2,
                opacity=0.7).add_to(m)

# Vis kartet
m



ModuleNotFoundError: No module named 'pandas'